**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Tue Jun  2 12:58:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
'Ahmad Beltagy (1).pdf'
'Ahmad Beltagy.pdf'
 annotations_trainval2014.zip
 annotations_trainval2017.zip
'[Beltagy] [4195] Polynomials'
 bert_hybrid
'BERT sequence classifier.ipynb'
 character_embedding.ipynb
'Colab Notebooks'
'Computer vision'
'Computer vision Cheat sheet.gdoc'
'Computer Vision Data'
'Computer Vision Project 5'
'Computer Vision Project Milestones.gdoc'
 darknet
 dev_labeled.csv
 Fine_tuning_using_script.ipynb
'Graduation Project'
 __MACOSX
 N

**1) Clone the Darknet**



In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 13618 (delta 0), reused 0 (delta 0), pack-reused 13615
Receiving objects: 100% (13618/13618), 12.24 MiB | 6.60 MiB/s, done.
Resolving deltas: 100% (9288/9288), done.


**2) Compile Darknet using Nvidia GPU**


In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1391:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^~~~
./src/image_opencv.cpp:1367:9: warning: unused variabl

**3) Configure Darknet network for training YOLO V3**

In [0]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [0]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [7]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [0]:
!echo "car" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [10]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-06-02 13:09:50--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.1’

darknet53.conv.74.1 100%[===================>] 154.96M   265KB/s    in 24m 20s 

2020-06-02 13:34:12 (109 KB/s) - ‘darknet53.conv.74.1’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [11]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/cars_008.jpg   
  inflating: data/obj/cars_008.txt   
  inflating: data/obj/cars_009.jpg   
  inflating: data/obj/cars_009.txt   
  inflating: data/obj/cars_010.jpg   
  inflating: data/obj/cars_010.txt   
  inflating: data/obj/cars_011.jpg   
  inflating: data/obj/cars_011.txt   
  inflating: data/obj/cars_012.jpg   
  inflating: data/obj/cars_012.txt   
  inflating: data/obj/cars_013.jpg   
  inflating: data/obj/cars_013.txt   
  inflating: data/obj/cars_014.jpg   
  inflating: data/obj/cars_014.txt   
  inflating: data/obj/classes.txt    
  inflating: data/obj/cars_001.jpg   
  inflating: data/obj/cars_001.txt   
  inflating: data/obj/cars_002.jpg   
  inflating: data/obj/cars_002.txt   
  inflating: data/obj/cars_003.jpg   
  inflating: data/obj/cars_003.txt   
  inflating: data/obj/cars_004.jpg   
  inflating: data/obj/cars_004.txt   
  inflating: data/obj/cars_005.jpg   
  inflating: data/obj/cars_005.txt   
  inflating: 

In [12]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

4 0.439556 0.588000 0.141333 0.062667

['4', '0.439556', '0.588000', '0.141333', '0.062667']
0 data/obj/cars_004.txt
0 0.439556 0.588000 0.141333 0.062667
4 0.459556 0.656667 0.156444 0.077333

['4', '0.459556', '0.656667', '0.156444', '0.077333']
0 data/obj/cars_004.txt
0 0.459556 0.656667 0.156444 0.077333
4 0.723111 0.586000 0.166222 0.077333

['4', '0.723111', '0.586000', '0.166222', '0.077333']
0 data/obj/cars_004.txt
0 0.723111 0.586000 0.166222 0.077333
4 0.727111 0.668333 0.152889 0.068667

['4', '0.727111', '0.668333', '0.152889', '0.068667']
0 data/obj/cars_004.txt
0 0.727111 0.668333 0.152889 0.068667
4 0.712444 0.439333 0.176889 0.078667

['4', '0.712444', '0.439333', '0.176889', '0.078667']
0 data/obj/cars_004.txt
0 0.712444 0.439333 0.176889 0.078667
4 0.388444 0.451333 0.158222 0.070667

['4', '0.388444', '0.451333', '0.158222', '0.070667']
0 data/obj/cars_004.txt
0 0.388444 0.451333 0.158222 0.070667
4 0.283556 0.526333 0.136889 0.064667

['4', '0.283556', '0.526333', '

In [13]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/cars_013.jpg', 'data/obj/cars_003.jpg', 'data/obj/cars_005.jpg', 'data/obj/cars_014.jpg', 'data/obj/cars_009.jpg', 'data/obj/cars_006.jpg', 'data/obj/cars_004.jpg', 'data/obj/cars_007.jpg', 'data/obj/cars_012.jpg', 'data/obj/cars_010.jpg', 'data/obj/cars_002.jpg', 'data/obj/cars_001.jpg', 'data/obj/cars_011.jpg', 'data/obj/cars_008.jpg']


In [0]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [0]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3_training
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x